In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline

#os.remove('/kaggle/working/housing.csv.2')
data='https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv'
!wget $data

df_org = pd.read_csv('housing.csv')

In [ ]:
(df_org.median_house_value.min(), df_org.median_house_value.max())

plt.figure(figsize=(10,10))
sns.histplot(df_org.median_house_value, bins=50)

In [ ]:
median_house_value_logs = np.log1p(df_org.median_house_value)
sns.histplot(median_house_value_logs, bins=30)

### Features
For the rest of the homework, you'll need to use only these columns:

* 'latitude',
* 'longitude',
* 'housing_median_age',
* 'total_rooms',
* 'total_bedrooms',
* 'population',
* 'households',
* 'median_income',
* 'median_house_value'

In [ ]:
columns = ['latitude','longitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income','median_house_value']
columns
df = df_org[columns].copy()

### Question 1
Find a feature with missing values. How many missing values does it have?

In [ ]:
df.isnull().sum()

### Question 2
What's the median (50% percentile) for variable 'population'?

In [ ]:
df.population.median()

### Split the data
* Shuffle the initial dataset, use seed 42.
* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Make sure that the target value ('median_house_value') is not in your dataframe.
* Apply the log transformation to the median_house_value variable using the np.log1p() function.

In [ ]:
n = len(df)
n_test = int(n * 0.2) # no. of elements for testing
n_valid = int(n * 0.2) # no. of elements for validation
n_train = n - n_valid - n_test # the rest if for training
(n, (n_train, n_valid, n_test), n_train + n_valid + n_test)

idx = np.arange(n)
idx
np.random.seed(42)
np.random.shuffle(idx)
idx

df_train = df.iloc[idx[:n_train]]
df_valid = df.iloc[idx[n_train: n_train + n_valid]]
df_test = df.iloc[idx[n_train + n_valid: ]]

df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

(len(df_train), len(df_valid), len(df_test))

# taregt vars in log1p form
y_train = np.log1p(df_train.median_house_value.values)
y_valid = np.log1p(df_valid.median_house_value.values)
y_test = np.log1p(df_test.median_house_value.values)

# remove target variable from the datasets
del df_train['median_house_value']
del df_valid['median_house_value']
del df_test['median_house_value']

### Question 3
* We need to deal with missing values for the column from Q1.
* We have two options: fill it with 0 or with the mean of this variable.
* Try both options. For each, train a linear regression model without regularization using the code from the lessons.
* For computing the mean, use the training only!
* Use the validation dataset to evaluate the models and compare the RMSE of each option.
* Round the RMSE scores to 2 decimal digits using round(score, 2)
* Which option gives better RMSE?

In [ ]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
    
    # Create Gram matrix
    XTX = X.T.dot(X)
    
    # Inverse the Gram matrix
    XTX_inv = np.linalg.inv(XTX)
    
    w_full = XTX_inv.dot(X.T).dot(y)
    return w_full[0], w_full[1:]

def get_rmse(y_pred, y):
    se = np.square( y_pred - y)
    mse = se.mean()
    return np.sqrt(mse)

#### Q3.1 Solution with 0 fill

In [ ]:
df_train.isna().sum()

# a df_train copy with na's filled with 0
df_train_zf = df_train.copy()
df_train_zf.total_bedrooms.fillna(0, inplace=True)

df_train.isna().sum()
df_train_zf.isna().sum()

# weights based on 0 filled training data
w0_zf, w_zf = train_linear_regression(df_train_zf, y_train)
(w0_zf, w_zf)

y_pred_zf = w0_zf + df_valid.dot(w_zf)

# RMSE for 0 filled based prediction 
round(get_rmse(y_pred_zf, y_valid), 2) # is 0.3295330365227974

#### Q3.2 Solution with mean fill

In [ ]:
df_train.isna().sum()

# a df_train copy with na's filled with mean of df_train.total_bedrooms
bdr_mean = df_train.total_bedrooms.mean()
df_train_mf = df_train.copy()
df_train_mf.total_bedrooms.fillna(bdr_mean, inplace=True)

df_train.isna().sum()
df_train_mf.isna().sum()

# weights based on 'mean' filled training data
w0_mf, w_mf = train_linear_regression(df_train_mf, y_train)
(w0_mf, w_mf)

y_pred_mf = w0_mf + df_valid.dot(w_mf)

# RMSE for 0 filled based prediction 
round(get_rmse(y_pred_mf, y_valid), 2) # is 0.3290195439006032

### Question 4
* Now let's train a regularized linear regression.
* For this question, fill the NAs with 0.
* Try different values of `r` from this list: `[0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]`.
* Use RMSE to evaluate the model on the validation dataset.
* Round the RMSE scores to 2 decimal digits.
* Which `r` gives the best RMSE?

In [ ]:
def train_linear_regression_reg(X, y, r=0.001):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
    
    # Create Gram matrix
    XTX = X.T.dot(X)
    XTX = XTX + r * np.eye(XTX.shape[0])
    
    # Inverse the Gram matrix
    XTX_inv = np.linalg.inv(XTX)
    
    w_full = XTX_inv.dot(X.T).dot(y)
    return w_full[0], w_full[1:]

In [ ]:
df_train_zf = df_train.copy()
df_train_zf.total_bedrooms.fillna(0, inplace=True)

r_candidates = [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]

for r in r_candidates:
    w0, w = train_linear_regression_reg(df_train_zf, y_train, r) #<---- this is training
    y_pred = w0 + df_valid.dot(w)
    print(('{:f}'.format(r), round(get_rmse(y_pred, y_valid),2)))

### Question 5
* We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.
* Try different seed values: `[0, 1, 2, 3, 4, 5, 6, 7, 8, 9].`
* For each seed, do the train/validation/test split with 60%/20%/20% distribution.
* Fill the missing values with 0 and train a model without regularization.
* For each seed, evaluate the model on the validation dataset and collect the RMSE scores.
* What's the standard deviation of all the scores? To compute the standard deviation, use `np.std`.
* Round the result to 3 decimal digits (`round(std, 3)`)

*Note: Standard deviation shows how different the values are. If it's low, then all values are approximately the same. If it's high, the values are different. If standard deviation of scores is low, then our model is stable.*

In [ ]:
n = len(df)
n_test = int(n * 0.2) # no. of elements for testing
n_valid = int(n * 0.2) # no. of elements for validation
n_train = n - n_valid - n_test # the rest if for training

rmse_results = []

for s in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:

    idx = np.arange(n)
    np.random.seed(s)
    np.random.shuffle(idx)

    df_train = df.iloc[idx[:n_train]]
    df_valid = df.iloc[idx[n_train: n_train + n_valid]]
    df_test = df.iloc[idx[n_train + n_valid: ]]

    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)

    y_train = np.log1p(df_train.median_house_value.values)
    y_valid = np.log1p(df_valid.median_house_value.values)
    y_test = np.log1p(df_test.median_house_value.values)

    # remove target variable from the datasets
    del df_train['median_house_value']
    del df_valid['median_house_value']
    del df_test['median_house_value']

    #---- 

    # a df_train copy with na's filled with 0
    df_train_zf = df_train.copy()
    df_train_zf.total_bedrooms.fillna(0, inplace=True)
    df_valid_zf = df_valid.copy()
    df_valid_zf.total_bedrooms.fillna(0, inplace=True)

    # weights based on 0 filled training data
    w0_zf, w_zf = train_linear_regression(df_train_zf, y_train)
    (w0_zf, w_zf)

    y_pred_zf = w0_zf + df_valid_zf.dot(w_zf)

    rmse_results.append(get_rmse(y_pred_zf, y_valid))

round(np.std(rmse_results), 3)

### Question 6
* Split the dataset like previously, use seed 9.
* Combine train and validation datasets.
* Fill the missing values with 0 and train a model with r=0.001.
What's the RMSE on the test dataset?

In [ ]:
n = len(df)
n_test = int(n * 0.2) # no. of elements for testing
n_valid = int(n * 0.2) # no. of elements for validation
n_train = n - n_valid - n_test # the rest if for training

idx = np.arange(n)
np.random.seed(9)
np.random.shuffle(idx)

df_train = df.iloc[idx[:n_train]]
df_valid = df.iloc[idx[n_train: n_train + n_valid]]
df_test = df.iloc[idx[n_train + n_valid: ]]

df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = np.log1p(df_train.median_house_value.values)
y_valid = np.log1p(df_valid.median_house_value.values)
y_test = np.log1p(df_test.median_house_value.values)

# remove target variable from the datasets
del df_train['median_house_value']
del df_valid['median_house_value']
del df_test['median_house_value']

#---- 

# a df_train copy with na's filled with 0
# df_train_zf = df_train.copy()
# df_valid_zf = df_valid.copy()
# df_train_zf.total_bedrooms.fillna(0, inplace=True)
# df_valid_zf.total_bedrooms.fillna(0, inplace=True)

df_test_zf = df_test.copy()
df_test_zf.total_bedrooms.fillna(0, inplace=True)
df_test_zf


df_train_full = pd.concat([df_train, df_valid])
df_train_full = df_train_full.reset_index(drop=True)
df_train_full.total_bedrooms.fillna(0, inplace=True)

y_train_full = np.concatenate([y_train, y_valid])

# # weights based on 0 filled training data
w0_zf, w_zf = train_linear_regression_reg(df_train_full, y_train_full, r=0.001)

y_pred_zf = w0_zf + df_test_zf.dot(w_zf)

round(get_rmse(y_pred_zf, y_test), 2)